In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import glob
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Load model

In [2]:
model = tf.keras.models.load_model("result/unet_mobilenet_128/model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


KeyboardInterrupt: 

In [ ]:
val_dir = os.path.join("data", "val_image", "class_0")
val_mask_dir = os.path.join("data", "val_mask", "class_0")

## Random pick a pic

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
w, h =  128, 128

In [ ]:
random_image_name = np.random.choice(glob.glob(os.path.join(val_dir, "*"))).split("/")[-1].replace(".jpg", "")
ori_img = cv2.imread(os.path.join(val_dir, random_image_name + ".jpg"))
mask_img = cv2.imread(os.path.join(val_mask_dir, random_image_name + ".png"))


plt.figure(figsize=(8, 5))
plt.title("Original")
plt.imshow(ori_img[:,:,::-1])
plt.axis('off')
plt.show()

plt.figure(figsize=(10, 8))
plt.subplot(221)
plt.title("Truth Mask")
mask_img[mask_img==0] = 255
mask_img[mask_img==1] = 0
mask_img[mask_img==2] = 180
plt.imshow(mask_img, cmap="gray")
plt.axis('off')


plt.subplot(222)
truth_img = ori_img.copy()
truth_img[mask_img==0] = 0
truth_img[mask_img==180] = 180
plt.title("Truth Result")
plt.imshow(truth_img[:,:,::-1])
plt.axis('off')


plt.subplot(223)
input_img = ori_img.copy()
input_img = cv2.resize(input_img, (w, h)) / 255.
pred = model.predict(np.expand_dims(input_img, axis=0))
pred = np.array(create_mask(pred)).reshape(w, h).astype('float32')

# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
# (thresh, binRed) = cv2.threshold(pred, 128, 255, cv2.THRESH_BINARY)
# pred = cv2.morphologyEx(pred, cv2.MORPH_OPEN, kernel, iterations=2)

pred[pred==0] = 255
pred[pred==1] = 0
pred[pred==2] = 180
pred = cv2.resize(pred, ori_img.shape[:2][::-1])
plt.title("Prediction Mask")
plt.imshow(pred, cmap="gray")
plt.axis('off')



plt.subplot(224)
# kernel = np.ones((5,5),np.uint8)  
# pred = cv2.dilate(pred, kernel,iterations=10)
# pred = cv2.erode(pred, kernel, iterations=3)
res_img = ori_img.copy()
res_img[pred==0] = 0
res_img[pred==180] = 180
plt.title("Prediction Result")
plt.imshow(res_img[:,:,::-1])
plt.axis('off')
plt.show()